In [1]:
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
from nltk.tokenize import RegexpTokenizer

In [2]:
df = pd.read_csv("D:/Year 3 Sem 1/Text Mining/Project B/Reviews.csv")

In [5]:
df.shape[0]

568454

In [30]:
df.tail()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...
568453,568454,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0,0,5,1338422400,Great Honey,"I am very satisfied ,product is as advertised,..."


In [6]:
dff = df.sample(n=600, random_state = 100)

In [7]:
dff.shape[0]

600

In [9]:
print(dff.dtypes)

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object


In [10]:
dff.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
238235,238236,B001E5E1O6,A2QVIP2O0J08BU,"Ashley H. Mattox ""lovedart""",0,0,5,1238889600,wow!,great product. great price for the product. ...
370991,370992,B000UXZ1PQ,A1CRI3DKT18JBX,D. Burck,2,2,5,1305072000,Fresh with consistent results,"Excellent value, with fresh grains, yielding c..."
147634,147635,B001D0IZBM,A2L24PZ59NJ3Q9,love to shop,0,0,5,1304208000,Fast shipping and excellent product,We enjoy the Keurig k-cup so much and there is...
393369,393370,B000W7PUOW,A2EC7QA31CFK4V,P. Tubb,2,2,5,1205971200,Planters Honey Peanuts,Iam very satisfied with the Planters honey pea...
399289,399290,B0039KER1G,A25RGKC4ZEZ3LV,jose lopez,5,7,1,1293753600,It is not mangosteen,It is not mangosteen.<br />I eat mangosteen fr...


In [11]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
dff['clean_msg']= dff['Text'].apply(lambda x:remove_punctuation(x))
#dff.head()

In [13]:
X = dff.clean_msg

In [18]:
#preprocess(dfNER.Text)

In [19]:
import nltk
import re
import spacy

In [20]:
grammar = "NP: {<DT>?<JJ>*<NN>}"
parser = nltk.RegexpParser(grammar)

In [21]:
def printTree(T):
    X = [x for x in T if isinstance(x, nltk.tree.Tree)]
    for n in X:
        return("---"+" ".join([x for (x,y) in n.leaves()]))

In [22]:
def NERregexp(text):
    reg = []
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    chunked = parser.parse(tagged)
    X = [x for x in chunked if isinstance(x, nltk.tree.Tree)]
    for n in X:
        reg.append(("---"+" ".join([x for (x,y) in n.leaves()])))
    
    return reg
    #printTree(chunked)    

In [23]:
def NERnltkClassifier(text):
    cl = []
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    chunked = nltk.ne_chunk(tagged)
    for chunk in chunked:
        if hasattr(chunk, 'label'):
            cl.append(("---"+chunk.label(), " ".join(c[0] for c in chunk)))
            #f = ("---"+chunk.label(), " ".join(c[0] for c in chunk))
            #print("---"+chunk.label(), " ".join(c[0] for c in chunk))
    return cl

In [24]:
nlp = spacy.load("en_core_web_sm")
def NERspacyClassifier(text):
    sp = []
    doc = nlp(text)
    for ent in doc.ents:
        sp.append(("---"+ent.label_, ent.text))
        #print("---"+ent.label_, ent.text)
    return sp

In [25]:
# for text in X:
#     print("----------------------------------------") 
#     print(text)
#     print("************REGEXP")
#     print(NERregexp(text))
#     print("************NLTK CLASSIFIER")
#     print(NERnltkClassifier(text))
#     print("************SPACY CLASSIFIER")
#     print(NERspacyClassifier(text))

In [26]:
output_df = pd.DataFrame(columns=['text', 'nerregexp', 'nernltkclassifier', 'nerspacyclassifier'])

In [27]:
for text in X:
    nerregexp_output = NERregexp(text) # This function does not return anything, you will need to create variable to store the output
    nernltkclassifier_output = NERnltkClassifier(text) # This function does not return anything, you will need to create variable to store the output
    nerspacyclassifier_output = NERspacyClassifier(text) # This function does not return anything, you will need to create variable to store the output
    output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

...

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'text': text, 'nerregexp': nerregexp_output, 'nernltkclassifier': nernltkclassifier_output, 'nerspacyclassifier': nerspacyclassifier_output}, ignore_index=True)
C:\Users\rolld\AppData\Local\Temp\ipykernel_19276\3596833997.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = 

In [28]:
output_df.head()

,text,nerregexp,nernltkclassifier,nerspacyclassifier
0,great product great price for the product qu...,"[---great product, ---great price, ---the prod...",[],[]
1,Excellent value with fresh grains yielding con...,"[---Excellent value, ---a risotto, ---cookbook...","[(---PERSON, Will)]",[]
2,We enjoy the Keurig kcup so much and there is ...,"[---a variety, ---husband, ---the dark]","[(---ORGANIZATION, Keurig), (---PERSON, Emeril...","[(---PERSON, Keurig), (---PRODUCT, Emerils Big..."
3,Iam very satisfied with the Planters honey pea...,"[---honey, ---second order, ---The price, ---t...","[(---PERSON, Iam), (---ORGANIZATION, Planters)...","[(---ORDINAL, second), (---PERSON, Patricia Tu..."
4,It is not mangosteenbr I eat mangosteen fresh ...,"[---mangosteen fresh fuit, ---anything, ---tha...","[(---GPE, Brazil)]","[(---GPE, Brazil)]"


In [29]:
import csv

def create_csv(X):
    # create a new CSV file
    with open('reviewNERv4.csv', 'w', newline='') as csvfile:
        # create a CSV writer object
        writer = csv.writer(csvfile)

        # write the header row
        writer.writerow(['Text', 'REGEXP', 'NLTK CLASSIFIER', 'SPACY CLASSIFIER'])

        # loop through the input texts
        for text in X:
            # apply the NER functions
            regexp_result = NERregexp(text)
            nltk_result = NERnltkClassifier(text)
            spacy_result = NERspacyClassifier(text)

            # write a row to the CSV file with the text and the results of the NER functions
            writer.writerow([text, regexp_result, nltk_result, spacy_result])

create_csv(X)